In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Assignment 2 Batch Pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
gsc_file_path = 'gs://de_2024_574440/data/data.csv'  
# Create data frame
df = spark.read.format("csv").option("header", "true") \
       .load(gsc_file_path)

df.printSchema()



root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)
 |-- releaseYear: string (nullable = true)



In [16]:
from pyspark.sql.functions import col, countDistinct

#Select only rows with movies or TvSeries
df = df.where((df.type=="movie") | (df.type=="tvSeries"))

In [17]:
from pyspark.sql.functions import split

#remove rows with null values for key columns
df = df.filter(df.averageRating != 'NULL') \
     .filter(df.genres != 'NULL') \
     .filter(df.releaseYear != 'NULL')

#transform strings into float and int
df = df.withColumn("averageRating", col("averageRating").cast("float")) \
    .withColumn("releaseYear", col("releaseYear").cast("integer")) \
    .withColumn("numVotes", col("numVotes").cast("integer"))


#Clean genres, so that only the main genre is available per row
df = df.withColumn("genre_t", split(df.genres, ', ')).drop("genres") \
     .withColumn("one_genre", col("genre_t").getItem(0)).drop("genre_t")

#Display the table
df.show()
              

+---------+--------------------+-----+-------------+--------+-----------+-----------+
|       id|               title| type|averageRating|numVotes|releaseYear|  one_genre|
+---------+--------------------+-----+-------------+--------+-----------+-----------+
|tt0000009|          Miss Jerry|movie|          5.4|     215|       1894|    Romance|
|tt0000147|The Corbett-Fitzs...|movie|          5.2|     539|       1897|Documentary|
|tt0000574|The Story of the ...|movie|          6.0|     941|       1906|     Action|
|tt0000591|    The Prodigal Son|movie|          5.7|      28|       1907|      Drama|
|tt0000615|  Robbery Under Arms|movie|          4.3|      27|       1907|      Drama|
|tt0000630|              Hamlet|movie|          3.1|      30|       1908|      Drama|
|tt0000675|         Don Quijote|movie|          4.2|      22|       1908|      Drama|
|tt0000679|The Fairylogue an...|movie|          5.2|      78|       1908|  Adventure|
|tt0000886|Hamlet, Prince of...|movie|          4.7|  

In [18]:
from pyspark.sql.functions import rank, row_number, desc
from pyspark.sql import Window

#Rank by genre 

window_genre=Window.partitionBy("one_genre").orderBy(desc("averageRating"))

df = df.withColumn("rank_genre", row_number().over(window_genre)) 

df.show()

+----------+--------------------+--------+-------------+--------+-----------+---------+----------+
|        id|               title|    type|averageRating|numVotes|releaseYear|one_genre|rank_genre|
+----------+--------------------+--------+-------------+--------+-----------+---------+----------+
| tt7156934|   Independent Roads|   movie|          9.9|      10|       2012|Adventure|         1|
| tt1053817|    Buried in Tucson|   movie|          9.8|      15|       2007|Adventure|         2|
| tt8114896|               Parto|   movie|          9.8|      85|       2010|Adventure|         3|
| tt1419921| Flying Over Everest|   movie|          9.7|      12|       2004|Adventure|         4|
|tt24224080|Jendela Seribu Su...|   movie|          9.7|      92|       2023|Adventure|         5|
|tt24228604|          OdBita Pot|tvSeries|          9.7|     874|       2022|Adventure|         6|
|tt33083593|        Vikaasaparva|   movie|          9.7|     505|       2024|Adventure|         7|
| tt982485

In [19]:
#Rank by both genre and year

window=Window.partitionBy("one_genre", "releaseYear").orderBy(desc("averageRating"))

df = df.withColumn("year_genre_rank", row_number().over(window)) 

df.show()

+---------+--------------------+-----+-------------+--------+-----------+---------+----------+---------------+
|       id|               title| type|averageRating|numVotes|releaseYear|one_genre|rank_genre|year_genre_rank|
+---------+--------------------+-----+-------------+--------+-----------+---------+----------+---------------+
|tt0000679|The Fairylogue an...|movie|          5.2|      78|       1908|Adventure|     13060|              1|
|tt0186564|      Sonho de Valsa|movie|          2.4|      24|       1909|Adventure|     16536|              1|
|tt0001184|Don Juan de Serra...|movie|          3.8|      22|       1910|Adventure|     15779|              1|
|tt0002130|     Dante's Inferno|movie|          7.0|    3662|       1911|Adventure|      4691|              1|
|tt0293219|Harry the Footballer|movie|          4.3|      33|       1911|Adventure|     15287|              2|
|tt0342837|Nankyoku tanken k...|movie|          6.8|      15|       1912|Adventure|      5837|              1|
|

In [20]:
# Use the Cloud Storage bucket for temporary BigQuery export data the connector uses.
bucket = "gs://de_2024_574440/temp_de2024"
spark.conf.set('temporaryGcsBucket', bucket)

df.write.format('bigquery') \
  .option('table', 'still-entity-435508-a1.Movies.IMDB_Comprehensive') \
  .mode("overwrite") \
  .save()